In [71]:
import requests
#from tqdm import tqdm
from pyspark.sql import SparkSession

In [72]:
#!pip install requests tqdm
#!pip install pyspark

In [73]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


In [74]:
sys.path

['C:\\Users\\DELL',
 'C:\\Users\\DELL\\AppData\\Local\\Temp\\spark-0cef82a8-0822-4c6e-b58c-b5603f43bcdd\\userFiles-2acffa45-3776-41ea-a849-1c81d9a2ee96',
 'C:\\anaconda\\python311.zip',
 'C:\\anaconda\\DLLs',
 'C:\\anaconda\\Lib',
 'C:\\anaconda',
 '',
 'C:\\anaconda\\Lib\\site-packages',
 'C:\\anaconda\\Lib\\site-packages\\win32',
 'C:\\anaconda\\Lib\\site-packages\\win32\\lib',
 'C:\\anaconda\\Lib\\site-packages\\Pythonwin']

In [75]:
!python --version

Python 3.11.4


In [76]:
# create pyspark session
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [10]:
import requests
from tqdm import tqdm

url = 'https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD'
response = requests.get(url, stream=True)

if response.status_code == 200:
    total_size = int(response.headers.get('content-length', 0))
    chunk_size = 1024  # You can adjust the chunk size as needed.
    
    with open('reported-crimes.csv', 'wb') as file, tqdm(
        desc='Downloading',
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=chunk_size):
            file.write(data)
            bar.update(len(data))
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

print("Download completed.")

Downloading: 1.73GB [22:50, 1.36MB/s]

Download completed.


In [77]:
# read first 5 ROWS
import pandas as pd
from pyspark.sql.functions import to_timestamp,col,lit
df = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

df.select('ID', 'Case Number', 'Date', 'IUCR').show(5)

+--------+-----------+-------------------+----+
|      ID|Case Number|               Date|IUCR|
+--------+-----------+-------------------+----+
|11646166|   JC213529|2018-09-01 00:01:00|0810|
|11645836|   JC212333|2016-05-01 00:25:00|1153|
|11243268|   JB167760|2017-01-01 00:01:00|1562|
| 1896258|    G749215|2001-12-15 02:00:00|0460|
|11645527|   JC212744|2015-02-02 10:00:00|1153|
+--------+-----------+-------------------+----+
only showing top 5 rows



In [78]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [79]:
# read with schema
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

labels = [
    ('ID', StringType()),
    ('Case Number', StringType()),
    ('Date', TimestampType()),
    ('Block', StringType()),
    ('IUCR', StringType()),
    ('Primary Type', StringType()),
    ('Description', StringType()),
    ('Location Description', StringType()),
    ('Arrest', StringType()),
    ('Domestic', BooleanType()),
    ('Beat', StringType()),
    ('Ward', StringType()),
    ('Community Area', StringType()),
    ('FBI Code', StringType()),
    ('X Coordinate', StringType()),
    ('Y Coordinate', StringType()),
    ('Year', IntegerType()),
    ('Updated On', TimestampType()),
    ('Latitude', DoubleType()),
    ('Longitude', DoubleType()),
    ('Location', StringType())
]

schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

StructType([StructField('ID', StringType(), True), StructField('Case Number', StringType(), True), StructField('Date', TimestampType(), True), StructField('Block', StringType(), True), StructField('IUCR', StringType(), True), StructField('Primary Type', StringType(), True), StructField('Description', StringType(), True), StructField('Location Description', StringType(), True), StructField('Arrest', StringType(), True), StructField('Domestic', BooleanType(), True), StructField('Beat', StringType(), True), StructField('Ward', StringType(), True), StructField('Community Area', StringType(), True), StructField('FBI Code', StringType(), True), StructField('X Coordinate', StringType(), True), StructField('Y Coordinate', StringType(), True), StructField('Year', IntegerType(), True), StructField('Updated On', TimestampType(), True), StructField('Latitude', DoubleType(), True), StructField('Longitude', DoubleType(), True), StructField('Location', StringType(), True)])

In [80]:
# read with schema

df = spark.read.csv('reported-crimes.csv', schema=schema, header=True)

df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: timestamp (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [81]:
df.select('ID', 'Case Number', 'Date', 'IUCR').show(5)

+--------+-----------+----+----+
|      ID|Case Number|Date|IUCR|
+--------+-----------+----+----+
|11646166|   JC213529|null|0810|
|11645836|   JC212333|null|1153|
|11243268|   JB167760|null|1562|
| 1896258|    G749215|null|0460|
|11645527|   JC212744|null|1153|
+--------+-----------+----+----+
only showing top 5 rows



In [82]:
# filter data 
df1 = df.filter(col('Year') >= 2022)
df1.select('ID', 'Case Number', 'Date', 'IUCR', 'year').show(5)

+--------+-----------+----+----+-------+
|      ID|Case Number|Date|IUCR|   year|
+--------+-----------+----+----+-------+
|10224881|   HY411873|null|1310|1875669|
|10225155|   HY412177|null|0470|1926785|
|10225206|   HY412253|null|041A|1873609|
|10225234|   HY412306|null|0420|1901570|
|10225308|   HY412341|null|2230|1924028|
+--------+-----------+----+----+-------+
only showing top 5 rows



In [83]:
# unique data

df.select('year').distinct().show()

+-------+
|   year|
+-------+
|1862793|
|1873451|
|1932760|
|1917570|
|1832283|
|1840869|
|1843917|
|1916160|
|1909736|
|1836937|
|1883862|
|1899958|
|1859559|
|1874811|
|1858095|
|1848641|
|1891458|
|1905902|
|1919587|
|1850347|
+-------+
only showing top 20 rows



In [52]:
# sorting
df.select('year').distinct().orderBy(col('year')).show()

+-------+
|   year|
+-------+
|   null|
|      0|
|1813894|
|1813896|
|1813897|
|1813908|
|1813909|
|1813910|
|1813911|
|1813913|
|1813914|
|1813917|
|1813919|
|1813925|
|1813930|
|1813931|
|1813932|
|1813937|
|1813938|
|1813939|
+-------+
only showing top 20 rows



In [84]:
# counts of the rows
df.count()

7878531

In [85]:
# grouped by counts
df.groupBy('Primary Type').count().orderBy('count', ascending = False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1662584|
|            BATTERY|1439063|
|    CRIMINAL DAMAGE| 898212|
|          NARCOTICS| 749291|
|            ASSAULT| 515453|
|      OTHER OFFENSE| 489070|
|           BURGLARY| 426930|
|MOTOR VEHICLE THEFT| 385705|
| DECEPTIVE PRACTICE| 351261|
|            ROBBERY| 296074|
+-------------------+-------+
only showing top 10 rows



Q: what percentage of reported crimes resulted in an arrest? 

In [86]:
# get groupwise counts
from pyspark.sql.functions import col, round

# Assuming df is your Spark DataFrame
grouped_df = df.groupBy('Arrest').count()

total_count = df.count()

ans = grouped_df.withColumn(
    'Percentage',
    round(col('count') / total_count * 100, 2).cast("double")
)

ans.show()

+------+-------+----------+
|Arrest|  count|Percentage|
+------+-------+----------+
| false|5832714|     74.03|
|  true|2045817|     25.97|
+------+-------+----------+



#### Output: 25.97%

Find top-3 location for reported crime

In [87]:
df.groupBy('Location Description').count().orderBy('count', ascending = False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|2054941|
|           RESIDENCE|1317288|
|           APARTMENT| 896269|
+--------------------+-------+
only showing top 3 rows



In [88]:
df.filter(col('Arrest') == True).count()

2045817

## Functions

In [89]:
# see all function
from pyspark.sql import functions
print(dir(functions))

['Any', 'ArrayType', 'Callable', 'Column', 'DataFrame', 'DataType', 'Dict', 'Iterable', 'JVMView', 'List', 'Optional', 'PandasUDFType', 'PySparkTypeError', 'PySparkValueError', 'PythonEvalType', 'SparkContext', 'StringType', 'StructType', 'TYPE_CHECKING', 'Tuple', 'Union', 'UserDefinedFunction', 'ValuesView', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_py_udf', '_from_numpy_type', '_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_columns', '_invoke_function_over_seq_of_columns', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_append', 'array_compact', 'array_contains', 'array_distinct', 'array_except', 'ar

#### Function Examples

In [90]:
# first 4 char of a string
from pyspark.sql.functions import lower, upper, substring

# get the documention 
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str: 'ColumnOrName', pos: int, len: int) -> pyspark.sql.column.Column
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. versionadded:: 1.5.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Notes
    -----
    The position is not zero based, but 1 based index.
    
    Parameters
    ----------
    str : :class:`~pyspark.sql.Column` or str
        target column to work on.
    pos : int
        starting position in str.
    len : int
        length of chars.
    
    Returns
    -------
    :class:`~pyspark.sql.Column`
        substring of given value.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]



In [91]:
# get the lower, upper, and first 4 characters of string
df.select(lower(col('Primary Type')), upper(col('Primary Type')), substring(col('Primary Type'), 1, 4).alias('PT-4')).show(5)

+-------------------+-------------------+----+
|lower(Primary Type)|upper(Primary Type)|PT-4|
+-------------------+-------------------+----+
|              theft|              THEFT|THEF|
| deceptive practice| DECEPTIVE PRACTICE|DECE|
|        sex offense|        SEX OFFENSE|SEX |
|            battery|            BATTERY|BATT|
| deceptive practice| DECEPTIVE PRACTICE|DECE|
+-------------------+-------------------+----+
only showing top 5 rows



In [92]:
# show the oldest date and most recent date
from pyspark.sql.functions import min, max, to_timestamp, col, lit

df = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

df.select('Date').show(5)

+-------------------+
|               Date|
+-------------------+
|2018-09-01 00:01:00|
|2016-05-01 00:25:00|
|2017-01-01 00:01:00|
|2001-12-15 02:00:00|
|2015-02-02 10:00:00|
+-------------------+
only showing top 5 rows



In [93]:
df.select(col('Date')).distinct().select(min(col('Date')).alias('Oldes Date'), max(col('Date')).alias('Recent Date')).show(1)

+-------------------+-------------------+
|         Oldes Date|        Recent Date|
+-------------------+-------------------+
|2001-01-01 00:00:00|2018-11-11 00:00:00|
+-------------------+-------------------+



In [98]:
new_df = spark.read.csv('reported-crimes.csv', header=True)

new_df.select('ID', 'Case Number', 'Date', 'IUCR').show(5, truncate=False)

+--------+-----------+----------------------+----+
|ID      |Case Number|Date                  |IUCR|
+--------+-----------+----------------------+----+
|11646166|JC213529   |09/01/2018 12:01:00 AM|0810|
|11645836|JC212333   |05/01/2016 12:25:00 AM|1153|
|11243268|JB167760   |01/01/2017 12:01:00 AM|1562|
|1896258 |G749215    |12/15/2001 02:00:00 AM|0460|
|11645527|JC212744   |02/02/2015 10:00:00 AM|1153|
+--------+-----------+----------------------+----+
only showing top 5 rows

